# IBM Applied Data Science Capstone Course by Coursera -Mahantesh Hiremath
### Week 5 Final Report
**_Opening a New Restaurant  in Bengaluru**
- Build a dataframe of neighborhoods in Bengaluruweb scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new shopping mall
***
### 1. Import libraries

In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [4]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore").text

In [5]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [6]:
# create a list to store neighborhood data
neighborhoodList = []

In [7]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [8]:
# create a new DataFrame from the list
blr_df = pd.DataFrame({"Neighborhood": neighborhoodList})

blr_df.head()

,Neighborhood
0,List of areas in Bangalore Cantonment
1,List of areas in Bengaluru Pete
2,List of neighbourhoods in Bangalore
3,Adugodi
4,"Agara, Bangalore"


In [9]:
# print the number of rows of the dataframe
blr_df.shape

(132, 1)

### 3. Get the geographical coordinates

In [10]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bangalore'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in blr_df["Neighborhood"].tolist() ]

In [12]:
coords

[[12.966180000000065, 77.58690000000007],
 [12.966180000000065, 77.58690000000007],
 [12.888130000000046, 77.57735000000008],
 [12.944020000000023, 77.60800000000006],
 [12.842830000000049, 77.48759000000007],
 [12.954080000000033, 77.54135000000008],
 [12.858110000000067, 77.55909000000008],
 [12.885680000000036, 77.59668000000005],
 [12.963480000000061, 77.61297000000008],
 [13.02753000000007, 77.65049000000005],
 [13.131820000000062, 77.67001000000005],
 [13.044710000000066, 77.55008000000004],
 [12.922310000000039, 77.56988000000007],
 [13.028477106077231, 77.63188968370176],
 [12.938980000000072, 77.57137000000006],
 [12.992220000000032, 77.53444000000007],
 [12.927350000000047, 77.67185000000006],
 [13.083010000000058, 77.54779000000008],
 [12.900090000000034, 77.60433000000006],
 [12.903080000000045, 77.62444000000005],
 [12.817530000000033, 77.67879000000005],
 [12.993330000000071, 77.66123000000005],
 [12.91488000000004, 77.61004000000008],
 [12.951940000000036, 77.54445000000

In [13]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [14]:
# merge the coordinates into the original dataframe
blr_df['Latitude'] = df_coords['Latitude']
blr_df['Longitude'] = df_coords['Longitude']

In [15]:
# check the neighborhoods and the coordinates
print(blr_df.shape)
blr_df

(132, 3)


,Neighborhood,Latitude,Longitude
0,List of areas in Bangalore Cantonment,12.966180,77.586900
1,List of areas in Bengaluru Pete,12.966180,77.586900
2,List of neighbourhoods in Bangalore,12.888130,77.577350
3,Adugodi,12.944020,77.608000
4,"Agara, Bangalore",12.842830,77.487590
5,Ananthnagar,12.954080,77.541350
6,Anjanapura,12.858110,77.559090
7,Arekere,12.885680,77.596680
8,Austin Town,12.963480,77.612970
9,Babusapalya,13.027530,77.650490


In [16]:
# save the DataFrame as CSV file
blr_df.to_csv("blr_df.csv", index=False)

### 4. Create a map of Bangalore with neighborhoods superimposed on top

In [19]:
# get the coordinates of Bangalore
address = 'Bangalore'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore{}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore12.9791198, 77.5912997.


In [20]:
# create map of Toronto using latitude and longitude values
map_blr = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(blr_df['Latitude'], blr_df['Longitude'], blr_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_blr

In [21]:
# save the map as HTML file
map_blr.save('map_blr.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [22]:
# define Foursquare Credentials and Version
CLIENT_ID = 'RRYRAZPFQENBCOLV51ULMXVT1DL4TIHI4JMY4V1MN2OAX2N2' # your Foursquare ID
CLIENT_SECRET = '0LHKSJEJLEAOHWHVFYPXYWVQZPAGFELWYYD1SN1QNDKQBDI2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RRYRAZPFQENBCOLV51ULMXVT1DL4TIHI4JMY4V1MN2OAX2N2
CLIENT_SECRET:0LHKSJEJLEAOHWHVFYPXYWVQZPAGFELWYYD1SN1QNDKQBDI2


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [23]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(blr_df['Latitude'], blr_df['Longitude'], blr_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [24]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(7055, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,List of areas in Bangalore Cantonment,12.96618,77.5869,Chikkana Tiffin Room,12.968557,77.584282,Indian Restaurant
1,List of areas in Bangalore Cantonment,12.96618,77.5869,ITC Gardenia,12.967010,77.595618,Hotel
2,List of areas in Bangalore Cantonment,12.96618,77.5869,Ravindra Kalakshetra,12.962176,77.584528,Theater
3,List of areas in Bangalore Cantonment,12.96618,77.5869,JW Marriott Hotel Bengaluru,12.972362,77.595051,Hotel
4,List of areas in Bangalore Cantonment,12.96618,77.5869,UB City,12.971709,77.595905,Shopping Mall


**Let's check how many venues were returned for each neighorhood**

In [25]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adugodi,100,100,100,100,100,100
Ananthnagar,26,26,26,26,26,26
Anjanapura,5,5,5,5,5,5
Arekere,76,76,76,76,76,76
Austin Town,100,100,100,100,100,100
BTM Layout,87,87,87,87,87,87
Babusapalya,50,50,50,50,50,50
"Bagalur, Bangalore Urban",4,4,4,4,4,4
Bahubalinagar,19,19,19,19,19,19


**Let's find out how many unique categories can be curated from all the returned venues**

In [26]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 236 uniques categories.


In [27]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Indian Restaurant', 'Hotel', 'Theater', 'Shopping Mall',
       'Fried Chicken Joint', 'Italian Restaurant',
       'South Indian Restaurant', 'Asian Restaurant', 'Lounge',
       'French Restaurant', 'Japanese Restaurant', 'Deli / Bodega',
       'Sushi Restaurant', 'Cupcake Shop', 'Brewery', 'Park', 'Bakery',
       'Vegetarian / Vegan Restaurant', 'Furniture / Home Store',
       'Cocktail Bar', 'Ice Cream Shop', 'Burger Joint',
       'Seafood Restaurant', 'Bed & Breakfast', 'Clothing Store',
       'Breakfast Spot', 'Wine Bar', 'Chinese Restaurant', 'Café', 'Pub',
       'Gym / Fitness Center', 'Athletics & Sports',
       'American Restaurant', 'Cricket Ground', 'Arcade',
       'Mexican Restaurant', 'Coffee Shop', 'Track Stadium',
       'Parsi Restaurant', 'Bubble Tea Shop', 'Botanical Garden',
       'Bagel Shop', 'Snack Place', 'Art Gallery', 'Movie Theater',
       'Boutique', 'Stadium', 'Steakhouse', 'Mediterranean Restaurant',
       'Capitol Building'], dtype=obje

In [28]:
# check if the results contain "Restaurant"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [31]:
# one hot encoding
blr_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
blr_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [blr_onehot.columns[-1]] + list(blr_onehot.columns[:-1])
blr_onehot = blr_onehot[fixed_columns]

print(blr_onehot.shape)
blr_onehot.head()

(7055, 237)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Inn,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Lighting Store,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Outdoor Supply Store,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Persian Restaurant,Pharmacy,Photography Lab,Photography Studio,Pizza Place,Platform,Plaza,Pool,Pool Hall,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Ramen Restaurant,Recreation Center,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Toll Booth,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,List of areas in Bangalore Cantonment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,List of areas in Bangalore Cantonment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [32]:
blr_grouped = blr_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(blr_grouped.shape)
blr_grouped

(131, 237)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Inn,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Lighting Store,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Outdoor Supply Store,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Persian Restaurant,Pharmacy,Photography Lab,Photography Studio,Pizza Place,Platform,Plaza,Pool,Pool Hall,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Ramen Restaurant,Recreation Center,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Toll Booth,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Adugodi,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.010000,0.010000,0.020000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070000,0.0,0.00000,0.000000,0.00,0.000000,0.000000,0.040000,0.00,0.010000,0.000000,0.030000,0.01,0.00000,0.000000,0.010000,0.000000,0.010000,0.00,0.00,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.00

In [34]:
len(blr_grouped[blr_grouped["Restaurant"] > 0])

71

**Create a new DataFrame for Restaurant data only**

In [35]:
blr_mall = blr_grouped[["Neighborhoods","Restaurant"]]

In [36]:
blr_mall.head()

,Neighborhoods,Restaurant
0,Adugodi,0.000000
1,Ananthnagar,0.038462
2,Anjanapura,0.000000
3,Arekere,0.013158
4,Austin Town,0.000000


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Bangalore into 3 clusters.

In [37]:
# set number of clusters
Bclusters = 3

blr_clustering = blr_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=Bclusters, random_state=0).fit(blr_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [38]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
blr_merged =blr_mall.copy()

# add clustering labels
blr_merged["Cluster Labels"] = kmeans.labels_

In [39]:
blr_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
blr_merged.head()

,Neighborhood,Restaurant,Cluster Labels
0,Adugodi,0.000000,0
1,Ananthnagar,0.038462,2
2,Anjanapura,0.000000,0
3,Arekere,0.013158,0
4,Austin Town,0.000000,0


In [40]:
# merge 
blr_merged = blr_merged.join(blr_df.set_index("Neighborhood"), on="Neighborhood")

print(blr_merged.shape)
blr_merged.head() # check the last columns!

(131, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Adugodi,0.000000,0,12.94402,77.60800
1,Ananthnagar,0.038462,2,12.95408,77.54135
2,Anjanapura,0.000000,0,12.85811,77.55909
3,Arekere,0.013158,0,12.88568,77.59668
4,Austin Town,0.000000,0,12.96348,77.61297


In [41]:
# sort the results by Cluster Labels
print(blr_merged.shape)
blr_merged.sort_values(["Cluster Labels"], inplace=True)
blr_merged

(131, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Adugodi,0.000000,0,12.944020,77.608000
82,Mariyannapalya,0.000000,0,13.049640,77.611820
81,Marathahalli,0.029412,0,12.954660,77.707520
80,Malleswaram,0.010000,0,13.006322,77.568416
79,Mahalakshmi Layout,0.017544,0,13.016350,77.544810
78,"Mahadevapura, Bangalore",0.000000,0,12.994090,77.666330
77,Madiwala,0.010000,0,12.920520,77.620900
76,List of neighbourhoods in Bangalore,0.028571,0,12.888130,77.577350
75,List of areas in Bengaluru Pete,0.010000,0,12.966180,77.586900
74,List of areas in Bangalore Cantonment,0.010000,0,12.966180,77.586900


**Finally, let's visualize the resulting clusters**

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(Bclusters)
ys = [i+x+(i*x)**2 for i in range(Bclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(blr_merged['Latitude'], blr_merged['Longitude'], blr_merged['Neighborhood'], blr_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [43]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [44]:
blr_merged.loc[blr_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Adugodi,0.000000,0,12.944020,77.608000
82,Mariyannapalya,0.000000,0,13.049640,77.611820
81,Marathahalli,0.029412,0,12.954660,77.707520
80,Malleswaram,0.010000,0,13.006322,77.568416
79,Mahalakshmi Layout,0.017544,0,13.016350,77.544810
78,"Mahadevapura, Bangalore",0.000000,0,12.994090,77.666330
77,Madiwala,0.010000,0,12.920520,77.620900
76,List of neighbourhoods in Bangalore,0.028571,0,12.888130,77.577350
75,List of areas in Bengaluru Pete,0.010000,0,12.966180,77.586900
74,List of areas in Bangalore Cantonment,0.010000,0,12.966180,77.586900


#### Cluster 1

In [46]:
blr_merged.loc[blr_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
122,Vidyaranyapura,0.111111,1,13.07927,77.55914
119,Uttarahalli,0.142857,1,12.89757,77.52830
113,Sulikunte,0.125000,1,12.88813,77.72807
21,Carmelaram,0.200000,1,12.91311,77.71565
14,Bharathnagar,0.111111,1,13.08301,77.54779


#### Cluster 2

In [45]:
blr_merged.loc[blr_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
123,"Vijayanagar, Bangalore",0.083333,2,13.076000,77.652380
116,Thindlu,0.090909,2,13.071270,77.566000
120,Varthur,0.066667,2,12.943480,77.747030
126,"Whitefield, Bangalore",0.073529,2,12.979380,77.733720
65,Kodigehalli,0.055556,2,13.067240,77.569920
109,"Siddapura, Bangalore",0.064516,2,12.956130,77.731960
93,Peenya,0.041667,2,13.031850,77.526790
88,Nagarbhavi,0.066667,2,12.956240,77.509360
86,Murugeshpalya,0.060241,2,12.955650,77.653350
72,Kundalahalli,0.048780,2,12.967520,77.715000


In [47]:
blr_merged.loc[blr_merged['Cluster Labels'] == 3]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude


#### Observations:
Most of the Restaurants are concentrated in the central area of Bengaluru city, with the highest number in cluster 2 and moderate number in cluster 0. On the other hand, cluster 1 has very low number to totally no Restaurant in the neighborhoods. This represents a great opportunity and high potential areas to open new Restaurant as there is very little to no competition from existing malls. Meanwhile, Restaurants in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of Restaurants.


From another perspective, this also shows that the oversupply of Restaurants mostly happened in the central area of the city, with the suburb area still have very few Restaurants. Therefore, this project recommends property developers to capitalize on these findings to open new Restaurants in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new Restaurants in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of Restaurants and suffering from intense competition.